GRAPHING FROM EXCEL FILES

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import ttest_ind
from scipy.stats import fisher_exact, chi2_contingency
from scipy.stats import t

In [ ]:
# Load the Excel file into a DataFrame
df = pd.read_excel("/Volumes/mohrlab/mohrlabspace/Nora_Mohr/ANALYSIS/Composite_Data/vRC_formation/vRC_formation.xlsx", sheet_name="All")  # Add sheet_name='Sheet1' if necessary
output_directory = "/Volumes/mohrlab/mohrlabspace/Nora_Mohr/ANALYSIS/Composite_Data/vRC_formation"
df.head()


In [ ]:
if 'file' in df.columns:
    if 'nucleus_label' in df.columns:
        df['unique_name'] = df['file'] + "_" + df['nucleus_label'].astype(str)
    else:
        print("Warning: 'nucleus_label' column missing")
        df['unique_name'] = df['file']

    if 'Experiment' in df.columns and 'Condition' in df.columns:
        df['unique_condition'] = df['Experiment'].astype(str) + '_' + df['Condition'].astype(str)
    else:
        print("Warning: 'Experiment' or 'Condition' column missing")
        df['unique_condition'] = df.get('Condition', 'Unknown')

    # Reorder columns: move 'unique_condition' and 'unique_name' to beginning
    cols = df.columns.tolist()
    for col in ['unique_condition', 'unique_name']:
        if col in cols:
            cols.remove(col)
    df = df[['unique_condition', 'unique_name'] + cols]
else:
    print("Warning: 'file' column missing — cannot create 'unique_name'")

df.head()



In [ ]:
subset_df = df[(df["volume_microns"] >= 500)]

output_path = os.path.join(output_directory, "vRC_data_NucVol-500.csv")
subset_df.to_csv(output_path, index=False)

In [ ]:
subset_df2 = df[(df["volume_microns"] >= 500) & (df["ICP8_volume"] > 0)]

output_path = os.path.join(output_directory, "vRC_data_NucVol-500_vRC-pos.csv")
subset_df2.to_csv(output_path, index=False)

In [ ]:
subset_df_py = df[(df["Valid"] == 1.0) & (df["volume_python"] > 0) & (df["volume_microns"] >= 500)]

output_path = os.path.join(output_directory, "vRC_data_NucVol-500_vRC-py-pos-valid.csv")
subset_df_py.to_csv(output_path, index=False)

In [ ]:
rep_counts = subset_df.groupby(['Condition', 'Timepoint'])['Experiment'].nunique().reset_index()
rep_counts = rep_counts.rename(columns={'Experiment': 'Replicate_Count'})
print(rep_counts)

In [ ]:
# Temporarily change display settings
pd.set_option('display.max_rows', None)

unique_combinations = subset_df[['Experiment', 'Timepoint', 'Condition']].drop_duplicates()
unique_combinations = unique_combinations.sort_values(by=['Condition', 'Timepoint', 'Experiment'])
print(unique_combinations)

# (Optional) Reset display option to default after printing
pd.reset_option('display.max_rows')

In [ ]:
custom_palette = {
    'HSV-1 (WT)': '#ff0066',        # blue
    'HSV-1 Confluent': '#7c0002',  # orange
    'HSV-1 + 150mM sorbitol': '#fc7bbc',   # green
    # Add more conditions as needed
}

print(df["Experiment"].unique())

In [ ]:
# Rename column for easier handling (optional)
subset_df = subset_df.rename(columns={'mean/background_ICP8': 'mean_background'})

# Group by condition and timepoint, and calculate median
median_df = subset_df.groupby(['Condition', 'Timepoint'])['mean_background'].median().reset_index()

# Plot
plt.figure(figsize=(10, 6))
sns.lineplot(
    data=median_df,
    x='Timepoint',
    y='mean_background',
    hue='Condition',
    marker='o',
    palette=custom_palette
)

plt.title('Median mean/background_ICP8 Over Time by Condition')
plt.ylabel('Median mean/background_ICP8')
plt.xlabel('Timepoint')
plt.legend(title='Condition')
plt.tight_layout()

plt.savefig(f"{output_directory}/median-mean_background_ICP8_all.png", bbox_inches="tight")
plt.show()


In [ ]:
# Rename column for easier handling (optional)
df = subset_df.rename(columns={'mean/background_ICP8': 'mean_background'})

# Group by condition and timepoint, and calculate median
median_df = subset_df.groupby(['unique_condition', 'Timepoint'])['mean_background'].median().reset_index()

# Line plot
plt.figure(figsize=(10, 6))
sns.lineplot(data=median_df, x='Timepoint', y='mean_background', hue='unique_condition', marker='o')

plt.title('Median mean/background_ICP8 Over Time by Experiment')
plt.ylabel('Median mean/background_ICP8')
plt.xlabel('Timepoint')
plt.legend(title='unique_condition')
plt.tight_layout()

plt.savefig(f"{output_directory}/median-mean_background_ICP8_all-experiments.png", bbox_inches="tight")

plt.show()


Remove unwanted timepoints/experiments that have errors

In [ ]:
remove_combinations = [
    ('241205', 12),
    ('250330', 24),
    # add more as needed
]

df_filtered = subset_df[~subset_df.apply(lambda row: (str(row['Experiment']), row['Timepoint']) in remove_combinations, axis=1)]
df_filtered_py = subset_df_py[~subset_df_py.apply(lambda row: (str(row['Experiment']), row['Timepoint']) in remove_combinations, axis=1)]

removed_df = subset_df[subset_df.apply(lambda row: (str(row['Experiment']), row['Timepoint']) in remove_combinations, axis=1)]
removed_df_py = subset_df_py[subset_df_py.apply(lambda row: (str(row['Experiment']), row['Timepoint']) in remove_combinations, axis=1)]

print("Rows removed:")
print(removed_df[['Experiment', 'Timepoint', 'Condition']].drop_duplicates())



In [ ]:
# Rename column for easier handling (optional)
df_filtered = df_filtered.rename(columns={'mean/background_ICP8': 'mean_background'})

# Group by condition and timepoint, and calculate median
median_df_filtered = df_filtered.groupby(['unique_condition', 'Timepoint'])['mean_background'].median().reset_index()

# Line plot
plt.figure(figsize=(10, 6))
sns.lineplot(data=median_df_filtered, x='Timepoint', y='mean_background', hue='unique_condition', marker='o')

plt.title('Median mean/background_ICP8 Over Time by Experiment')
plt.ylabel('Median mean/background_ICP8')
plt.xlabel('Timepoint')
plt.legend(title='unique_condition')
plt.tight_layout()
plt.show()

In [ ]:
# Ensure Timepoint is numeric
df_filtered['Timepoint'] = pd.to_numeric(df_filtered['Timepoint'], errors='coerce')

# Rename column for easier handling (optional)
df_filtered = df_filtered.rename(columns={'mean/background_ICP8': 'mean_background'})

# Group by condition and timepoint, and calculate median
median_df_filtered = df_filtered.groupby(['Condition', 'Timepoint'])['mean_background'].median().reset_index()

# Line plot
plt.figure(figsize=(10, 6))
sns.lineplot(data=median_df_filtered, x='Timepoint', y='mean_background', hue='Condition', marker='o', palette=custom_palette)

plt.title('Median mean/background_ICP8 Over Time by Condition')
plt.ylabel('Median mean/background_ICP8')
plt.xlabel('Timepoint')
plt.legend(title='Condition')
plt.tight_layout()

plt.savefig(f"{output_directory}/median-mean_background_ICP8_filtered.png", bbox_inches="tight")

plt.show()


In [ ]:
#More paper-friendly figure of Median mean/background:
# Ensure Timepoint is numeric
df_filtered['Timepoint'] = pd.to_numeric(df_filtered['Timepoint'], errors='coerce')

# Save the filtered DataFrame to CSV before renaming columns
df_filtered.to_csv(f"{output_directory}/filtered_conditions_n-3.csv", index=False)

# Rename column for easier handling (optional)
df_filtered = df_filtered.rename(columns={'mean/background_ICP8': 'mean_background'})

# Group by condition and timepoint, and calculate median
median_df_filtered = df_filtered.groupby(['Condition', 'Timepoint'])['mean_background'].median().reset_index()

# Line plot
plt.figure(figsize=(6, 6))
sns.lineplot(data=median_df_filtered, x='Timepoint', y='mean_background', hue='Condition', marker='o', linewidth=3, markersize=8, palette=custom_palette,)

plt.title('Median mean/background_ICP8 Over Time by Condition')
plt.ylabel('Median mean/background_ICP8')
plt.xlabel('Timepoint')
plt.legend(title='Condition')
plt.tight_layout()

plt.savefig(f"{output_directory}/median-mean_background_ICP8_filtered_PAPER.png", bbox_inches="tight")
plt.show()

In [ ]:
#THIS ONE HAS EXPERIMENTAL REPLICATES & STATS FOR THE PAPER-FRIENDLY FIGURE of Median mean/background:
# Ensure Timepoint is numeric
df_filtered['Timepoint'] = pd.to_numeric(df_filtered['Timepoint'], errors='coerce')

# Save the filtered DataFrame to CSV before renaming columns
df_filtered.to_csv(f"{output_directory}/filtered_conditions_n-3.csv", index=False)

# Rename column for easier handling (optional)
df_filtered = df_filtered.rename(columns={'mean/background_ICP8': 'mean_background'})

# Compute medians at the replicate level
replicate_medians = (
    df_filtered
    .groupby(['Experiment', 'Condition', 'Timepoint'])['mean_background']
    .median()
    .reset_index()
)

# Save median per replicate
replicate_medians.to_csv(f"{output_directory}/replicate_medians.csv", index=False)

# Line plot
plt.figure(figsize=(6, 6))
sns.lineplot(data=replicate_medians, x='Timepoint', y='mean_background', hue='Condition', errorbar=('ci', 95), marker='o', linewidth=3, markersize=8, palette=custom_palette,)

plt.title('Median mean/background_ICP8 Over Time by Condition')
plt.ylabel('Median mean/background_ICP8')
plt.xlabel('Timepoint')
plt.legend(title='Condition')
plt.tight_layout()

plt.savefig(f"{output_directory}/median-mean_background_ICP8_filtered_PAPER_CI95.png", bbox_inches="tight")
plt.show()

stat_results = []

for timepoint in sorted(replicate_medians['Timepoint'].unique()):
    subset = replicate_medians[replicate_medians['Timepoint'] == timepoint]
    groups = subset.groupby('Condition')['mean_background'].apply(list)

    if len(groups) == 2:
        cond1, cond2 = groups.index
        stat, pval = stats.mannwhitneyu(groups[cond1], groups[cond2], alternative='two-sided')
        test_name = "Mann-Whitney U"
    else:
        stat, pval = stats.kruskal(*groups)
        test_name = "Kruskal-Wallis"

    stat_results.append({
        "Timepoint": timepoint,
        "Test": test_name,
        "Statistic": stat,
        "p-value": pval
    })

# Save statistical results
stat_results_df = pd.DataFrame(stat_results)
stat_results_df.to_csv(f"{output_directory}/median-mean_background_ICP8_filtered_CI95_statistical_tests.csv", index=False)

In [ ]:
#THIS ONE IS MEDIAN OF MEDIANS mean/background:

# Grand median across experiments
summary_median_df = (
    replicate_medians
    .groupby(['Condition', 'Timepoint'])['mean_background']
    .median()
    .reset_index()
)

# Line plot
plt.figure(figsize=(6, 6))
sns.lineplot(data=summary_median_df, x='Timepoint', y='mean_background', hue='Condition', errorbar=('ci', 95), marker='o', linewidth=3, markersize=8, palette=custom_palette,)

plt.title('Median mean/background_ICP8 Over Time by Condition')
plt.ylabel('Median mean/background_ICP8')
plt.xlabel('Timepoint')
plt.legend(title='Condition')
plt.tight_layout()

plt.savefig(f"{output_directory}/median-of-medians-mean_background_ICP8_filtered_PAPER.png", bbox_inches="tight")
plt.show()


In [ ]:
#ABOVE PLOT WITH ERROR BARS/COMPARISONS
#Compute grand median and 95% CI
# 2. Compute grand median and 95% CI
summary_stats = []
for (condition, timepoint), group in replicate_medians.groupby(['Condition', 'Timepoint']):
    values = group['mean_background'].values
    n = len(values)
    median_val = np.median(values)
    sem = np.std(values, ddof=1) / np.sqrt(n) if n > 1 else 0
    ci95 = sem * t.ppf(0.975, df=n-1) if n > 1 else 0
    summary_stats.append({
        'Condition': condition,
        'Timepoint': timepoint,
        'median': median_val,
        'ci_lower': median_val - ci95,
        'ci_upper': median_val + ci95
    })

grand_median_df = pd.DataFrame(summary_stats)
grand_median_df.to_csv(f"{output_directory}/median-of-medians-mean_background_ICP8_filtered_CI95_statistical_tests.csv", index=False)

# 3. Plot using error bars manually
plt.figure(figsize=(6, 6))
for condition, group in grand_median_df.groupby("Condition"):
    color = custom_palette.get(condition, "#333333")
    
    plt.errorbar(
        group['Timepoint'],
        group['median'],
        yerr=[group['median'] - group['ci_lower'], group['ci_upper'] - group['median']],
        fmt='o-',
        capsize=4,
        linewidth=3,
        markersize=8,
        label=condition,
        color=color,
        ecolor=color
    )

plt.title('Median of Replicate Medians with 95% CI')
plt.ylabel('Median mean/background_ICP8')
plt.xlabel('Timepoint')
plt.legend(title='Condition')
plt.tight_layout()
plt.savefig(f"{output_directory}/median-of-medians-mean_background_ICP8_filtered_PAPER_CI95.png", bbox_inches="tight")
plt.show()


In [ ]:
# Ensure Timepoint is numeric
df_filtered['Timepoint'] = pd.to_numeric(df_filtered['Timepoint'], errors='coerce')

# Group by condition and timepoint, calculate mean and SEM for ICP8_volume
summary_df = df_filtered.groupby(['Condition', 'Timepoint'])['ICP8_volume'].agg(
    mean='mean',
    sem=lambda x: np.std(x, ddof=1) / np.sqrt(len(x))
).reset_index()

# Filter for timepoints >= 9
summary_df = summary_df[summary_df['Timepoint'] >= 9]

# Plot with error bars
plt.figure(figsize=(10, 6))
for condition, group in summary_df.groupby('Condition'):
    plt.errorbar(
        group['Timepoint'], 
        group['mean'], 
        yerr=group['sem'], 
        label=condition, 
        marker='o', 
        capsize=4,
        color=custom_palette.get(condition, '#333333')
    )

plt.title('Mean vRC Volume Over Time by Condition (Timepoints ≥ 9)')
plt.ylabel('Mean vRC Volume (n.u.)')
plt.xlabel('Timepoint')
plt.legend(title='Condition')
plt.tight_layout()

plt.savefig(f"{output_directory}/vRC_volume_filtered.png", bbox_inches="tight")
plt.show()

In [ ]:
# Ensure Timepoint is numeric
df_filtered_py['Timepoint'] = pd.to_numeric(df_filtered_py['Timepoint'], errors='coerce')

# Group by condition and timepoint, calculate mean and SEM for ICP8_volume
summary_df_py = df_filtered_py.groupby(['Condition', 'Timepoint'])['volume_python'].agg(
    mean='mean',
    sem=lambda x: np.std(x, ddof=1) / np.sqrt(len(x))
).reset_index()

# Filter for timepoints >= 9
summary_df_py = summary_df_py[summary_df_py['Timepoint'] >= 9]

# Plot with error bars
plt.figure(figsize=(10, 6))
for condition, group in summary_df_py.groupby('Condition'):
    plt.errorbar(
        group['Timepoint'], 
        group['mean'], 
        yerr=group['sem'], 
        label=condition, 
        marker='o', 
        capsize=4,
        color=custom_palette.get(condition, '#333333')
    )

plt.title('Mean vRC Volume Over Time by Condition (Timepoints ≥ 9)')
plt.ylabel('Mean vRC Volume (n.u.)')
plt.xlabel('Timepoint')
plt.legend(title='Condition')
plt.tight_layout()

plt.savefig(f"{output_directory}/vRC_volume_filtered_py.png", bbox_inches="tight")
plt.show()